Your task is to train *character-level* language models. 
You will train unigram, bigram, and trigram character-level models on a collection of books from Project Gutenberg. You will then use these trained English language models to distinguish English documents from Brazilian Portuguese documents in the test set.

In [28]:
import pandas as pd
import httpimport
import sklearn_pandas
from sklearn.model_selection import train_test_split
with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

This code loads the training and test data. Each dataset is a list of books. Each book contains a list of sentences, and each sentence contains a list of words. For building a character language model, you should join the words of a sentence together with a space character.

In [55]:
# get the train and test data
train = get_train_data()
test, test_files = get_test_data()
train,held_out  = train_test_split(train, test_size=0.20)

## 1.1
Collect statistics on the unigram, bigram, and trigram character counts.

If your machine takes a long time to perform this computation, you may save these counts to files in your github repository and load them on request. This is not necessary, however.

In [31]:
import re
def sentences(train):
  train_edit=[]
  for i in train:
    for j in i:
      sentence=" ".join(j)
      sentence=sentence.lower()
      sentence = re.sub(r'[^A-Za-z. ]', '', sentence)
      train_edit.append(sentence)
  return train_edit

train_edit=sentences(train)
print("train_edit",train_edit[0])
########################################
def grams(n,train_edit):
  gram_set=[]
  for i in train_edit:
    for j in range(len(i)-n+1):
      if n==1:
        gram_set.append(i[j:j+n])
      elif n==2:
        gram_set.append(i[j:j+n])
      else:
        gram_set.append(i[j:j+n])
  return gram_set

train_edit  the tragedie of macbeth by william shakespeare  


In [32]:
unigram=grams(1,train_edit)
bigram=grams(2,train_edit)
trigram=grams(3,train_edit)

for a in range(0,10):
  print(trigram[a])
# doing well

 th
the
he 
e t
 tr
tra
rag
age
ged
edi


In [33]:
def stats(grams):
  dict={}
  for item in grams:
    if (item in dict):
      dict[item] += 1
    else:
      dict[item] = 1
  return dict

In [34]:
unigram_dict=stats(unigram)
bigram_dict=stats(bigram)
trigram_dict=stats(trigram)

In [35]:
print(trigram_dict.keys())
if 'man' in trigram_dict:
  print("yes")

dict_keys([' th', 'the', 'he ', 'e t', ' tr', 'tra', 'rag', 'age', 'ged', 'edi', 'die', 'ie ', 'e o', ' of', 'of ', 'f m', ' ma', 'mac', 'acb', 'cbe', 'bet', 'eth', 'th ', 'h b', ' by', 'by ', 'y w', ' wi', 'wil', 'ill', 'lli', 'lia', 'iam', 'am ', 'm s', ' sh', 'sha', 'hak', 'ake', 'kes', 'esp', 'spe', 'pea', 'ear', 'are', 're ', 'e  ', 'act', 'ctu', 'tus', 'us ', 's p', ' pr', 'pri', 'rim', 'imu', 'mus', 's .', 'sco', 'coe', 'oen', 'ena', 'na ', 'a p', 'ima', 'ma ', 'a .', 'thu', 'hun', 'und', 'nde', 'der', 'er ', 'r a', ' an', 'and', 'nd ', 'd l', ' li', 'lig', 'igh', 'ght', 'htn', 'tni', 'nin', 'ing', 'ng ', 'g .', 'ent', 'nte', 'ter', 'r t', 'thr', 'hre', 'ree', 'ee ', 'e w', 'wit', 'itc', 'tch', 'che', 'hes', 'es ', 'whe', 'hen', 'en ', 'n s', 'hal', 'all', 'll ', 'l w', ' we', 'we ', 'e m', ' me', 'mee', 'eet', 'et ', 't a', ' ag', 'aga', 'gai', 'ain', 'ine', 'ne ', 'in ', 'n t', 'r  ', '  l', 'g  ', '  o', ' or', 'or ', 'r i', ' in', 'n r', ' ra', 'rai', 'e h', ' hu', 'hur', 'u

In [36]:
import numpy as np

lambda_val=[[0.7,0.2,0.0999,0.0001],
            [0.7,0.15,0.1499,0.0001],
            [0.8,0.1,0.0999,0.0001],
            [0.6,0.2,0.1999,0.0001],
            [0.82,0.1,0.0799,0.0001],
            [0.85,0.1,0.0499,0.0001],
            [0.85,0.12,0.0299,0.0001],
            [0.9,0.05,0.0499,0.0001],
            [0.9,0.08,0.0199,0.0001],
            [0.95,0.03,0.0199,0.0001]]
for i in lambda_val:
  print(np.sum(i))
print(np.sum(lambda_val, axis=1))

0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [42]:
import math
def perplexity_with_interpolation(lamb,sentence):
  probs=0.0
  count=0
  for i in range(len(sentence)-2):
    token= sentence[i:i+3]
    max_likelyhood=0
    if token in trigram_dict.keys():
      max_likelyhood+=lamb[0]*trigram_dict[token]/bigram_dict[token[:-1]]
    token =token[:-1]
    if token in bigram_dict.keys():
      max_likelyhood+=lamb[1]*bigram_dict[token]/unigram_dict[token[:-1]]
    token=token[:-1]
    if token in unigram_dict.keys():
      max_likelyhood+= lamb[2]*unigram_dict[token]/(sum(unigram_dict.values()))
    else:
      max_likelyhood+= lamb[3]/(sum(unigram_dict.values()))
    max_likelyhood=math.log(max_likelyhood)#log of likelyhood after interpolating linearly with 4 lambdas lamb
    probs=probs + max_likelyhood
    count+=1
  perplexity=2**(-1*probs/count)
  return perplexity  

In [50]:
held_out_sentences=sentences(held_out)
print("sentences", held_out_sentences)
#print("".join(held_out_sentences))


sentences [' the man who was thursday by g . k . chesterton  ', 'to edmund clerihew bentley', 'a cloud was on the mind of men  and wailing went the weather  yea  a sick cloud upon the soul when we were boys together .', 'science announced nonentity and art admired decay  the world was old and ended  but you and i were gay  round us in antic order their crippled vices came  lust that had lost its laughter  fear that had lost its shame .', 'like the white lock of whistler  that lit our aimless gloom  men showed their own white feather as proudly as a plume .', 'life was a fly that faded  and death a drone that stung  the world was very old indeed when you and i were young .', 'they twisted even decent sin to shapes not to be named  men were ashamed of honour  but we were not ashamed .', 'weak if we were and foolish  not thus we failed  not thus  when that black baal blocked the heavens he had no hymns from us children we were  our forts of sand were even as weak as eve  high as they went

In [48]:
#Best Lambda
Perp_val_for_lamb=[]
for i in lambda_val:
  Perp_val=perplexity_with_interpolation(i,"".join(held_out_sentences))
  print(i," Perplexity with this lambda set is : ", Perp_val)
  Perp_val_for_lamb.append(Perp_val)
best_lambda=lambda_val[Perp_val_for_lamb.index(min(Perp_val_for_lamb))]
print("Best lambda value set: ",best_lambda)


[0.7, 0.2, 0.0999, 0.0001]  Perplexity with this lambda set is :  3.634899554130262
[0.7, 0.15, 0.1499, 0.0001]  Perplexity with this lambda set is :  3.694037222484275
[0.8, 0.1, 0.0999, 0.0001]  Perplexity with this lambda set is :  3.7265516075402014
[0.6, 0.2, 0.1999, 0.0001]  Perplexity with this lambda set is :  3.7029366314568
[0.82, 0.1, 0.0799, 0.0001]  Perplexity with this lambda set is :  3.722901434048391
[0.85, 0.1, 0.0499, 0.0001]  Perplexity with this lambda set is :  3.7237809340743455
[0.85, 0.12, 0.0299, 0.0001]  Perplexity with this lambda set is :  3.701309216431086
[0.9, 0.05, 0.0499, 0.0001]  Perplexity with this lambda set is :  3.822556568645272
[0.9, 0.08, 0.0199, 0.0001]  Perplexity with this lambda set is :  3.7760314960423598
[0.95, 0.03, 0.0199, 0.0001]  Perplexity with this lambda set is :  3.9075624526243535
Best lambda value set:  [0.7, 0.2, 0.0999, 0.0001]


In [123]:
print(len(test))
print(len(test_files),len(test_sentences))
test_data=[]
for i in range(0,len(test)):
  total=[]
  for j in test[i]:
    total += j
  total=" ".join(total)
  total= re.sub(r'[^A-Za-z. ]', '', total) 
  test_data.append(total)

for i in range(0,10):
  print(test_data[i])
  print(len(test_data))

220
220 28237
The long and everincreasing column of sportsmen is now moving into a new era . Modern times have changed the world beyond recognition . The early years of the twentieth century seem very far away . But with all the changes in philosophy  dress and terrain  a few things remain constant  including the devotion of Americans to the great field sports  hunting and fishing . As the generations move on  clothes become more suitable for the enjoyment of outdoor sports . Sporting firearms change  markedly for the better . Just as modern transportation has outmoded the early Studebaker covered wagon  the demand of todays sportsmen and women has necessitated changes in their equipment . The American firearms and ammunition manufacturers through diligent research and technical development have replaced the muzzle loader and slowfiring singleshot arms with modern fast firing autoloaders  extremely accurate bolt  lever  and slide action firearms . And millions of rounds of entirely new

In [85]:
#test_sentences=sentences(test)
#test_data="".join(held_out_sentences)
test_file_perp={}
for i in range(0,len(test_data)):
  Perp_val=perplexity_with_interpolation(best_lambda,test_data[i])
  test_file_perp[test_files[i]]=Perp_val
  #print(test_files[i]," : ", Perp_val)
sorted_perp_test_files = dict( sorted(test_file_perp.items(),
                           key=lambda item: item[1],
                           reverse=False))
#print('Sorted Dictionary: ')
print(sorted_perp_test_files)


{'cd11': 4.243099488158383, 'cf27': 4.331326555355117, 'cf39': 4.418697185133757, 'cf26': 4.4213784476538684, 'cf07': 4.4448279980833645, 'cg11': 4.464969237587936, 'cd01': 4.4874963466360684, 'cf35': 4.499399419116064, 'cf44': 4.511509826137276, 'ce17': 4.519148094064781, 'cf48': 4.521452961170676, 'ce28': 4.545708820116075, 'ce21': 4.587118650097594, 'cf13': 4.58987326493591, 'ce19': 4.592473310372482, 'cd02': 4.639415529126726, 'cf12': 4.653744597441389, 'cg05': 4.668867587967658, 'ca35': 4.6697129772614066, 'ce29': 4.670549822331218, 'ce24': 4.676105263726082, 'cg09': 4.683659550492977, 'cf01': 4.688879743183828, 'cd13': 4.709360778048159, 'cd12': 4.716655894699798, 'cg04': 4.731348047439646, 'cf03': 4.733870866415028, 'ce08': 4.7576841432675145, 'cf33': 4.771243170900655, 'cf43': 4.774593484301992, 'ce02': 4.831984935280168, 'ce20': 4.881513884936232, 'cg02': 4.892350395131519, 'cf11': 4.903799830799333, 'cf37': 4.91755595162035, 'ce35': 4.931057365802203, 'cd04': 4.94824127756970

In [103]:
print(test_data[test_files.index("ce09")])
#threshold=9.09, higher or equal to 9.09 is Brazilian file

Orlando  Fla.  Feb.   The best yearold pacing mile up to date at Ben White Raceway has been that of Mary Liner  MainlinerHighland Ellen   a member of the Dick Williams stable  who was clocked  . She is owned by Ralph H. Kroening  Milwaukee  Wis.  who  according to the railbirds  can feel justly proud of her . Other good miles have been by Debonnie  Dale FrostDebby Hanover  and Prompt Time  AdiosOn Time  in    Kimberly Gal  GalophoneKimberly Hanover  .   Laguerre Hanover  Tar HeelLotus Hanover  and Monel  Tar HeelMiracle Byrd  in  . Laguerre Hanover is outstanding in type and conformation  good body  plenty of heart girth  stands straight on his legs on excellent feet  and has the smoothest gait . This colt is behind most of the other yearolds in the Simpson stable but can show about as much pace as any of them . Monel shows improvement with each workout and looks the makings of a good brood mare after winning her share of races . Stardel  Stars PrideStarlette Hanover     looks quite pr

## 1.2
Calculate the perplexity for each document in the test set using the linear interpolation smoothing method. For determining λs for linear interpolation, you can divide the training data into a new training set (80%) and a held-out set (20%), then using grid search method:
Choose ~10 values of λ to test using grid search on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows: 
  - Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names (from `test_files`) and perplexities of the documents above the threshold

    ```
        file name, score
        file name, score
        . . .
        file name, score
    ```

  - Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.




In [105]:
# accuracy
threshold_val_perp=9.09
def accuracy(sorted_perp_test_files,threshold_val_perp):
  correct=0
  wrong=0
  for k,v in sorted_perp_test_files.items():
    if v >threshold_val_perp:
      if re.search("txt",k):
        correct+=1
      else:
        wrong+=1
    else:
      if re.search("txt",k):
        wrong+=1
      else:
        correct+=1
  acc=round((correct*100)/(correct+wrong),2)
  return acc


In [106]:
print(accuracy(sorted_perp_test_files,9.09))

97.27


## 1.3
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

  - Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names and perplexities of the documents above the threshold

  ```
      file name, score
      file name, score
      . . .
      file name, score
  ```

  - Copy this list of filenames and manually annotate them for correctness.

In [124]:
#perplexity with lambda=0.1
def perplexity_add_k_smoothing(k,sentence):
  probs=0.0
  count=0
  max_likelyhood=0.0
  for i in range(len(sentence)-2):
    token= sentence[i:i+3]
    #for k in trigram_dict.values():
    max_likelyhood=(trigram_dict.get(token,0.0) + k) / (bigram_dict.get(token[:-1],0.0)+ k*len(trigram_dict))
    max_likelyhood=math.log(max_likelyhood)#log of likelyhood after interpolating linearly with 4 lambdas lamb
    probs=probs + max_likelyhood
    count+=1
  perplexity=2**(-1*probs/count)
  return perplexity  

In [130]:
test_file_perp={}
for i in range(0,len(test_data)):
  Perp_val=perplexity_add_k_smoothing(0.1,test_data[i])
  test_file_perp[test_files[i]]=Perp_val
  #print(test_files[i]," : ", Perp_val)
sorted_perp_test_files = dict( sorted(test_file_perp.items(),
                           key=lambda item: item[1],
                           reverse=False))
#print('Sorted Dictionary: ')
print(sorted_perp_test_files)


{'cd11': 4.76533405805131, 'cf27': 4.839117402437304, 'cd01': 4.923623448601809, 'cf26': 4.924847098878527, 'cg11': 4.988290949861411, 'cf07': 4.99953532491482, 'ce17': 5.008218328229685, 'cf48': 5.069752747418721, 'cf12': 5.073215470437581, 'ce21': 5.085094625433438, 'cf13': 5.091568005916731, 'cf35': 5.10180902596074, 'cf39': 5.110576065209118, 'cf44': 5.120722598894697, 'ce19': 5.153992921863663, 'ce28': 5.1858560812405265, 'cg04': 5.196375047278154, 'ce29': 5.205534090883376, 'ce24': 5.212947506558583, 'cg09': 5.222913248590429, 'cd13': 5.234800616968562, 'cd02': 5.2380485001103185, 'cg05': 5.256727673524201, 'ca35': 5.265994482585753, 'cd12': 5.267452501392718, 'ce08': 5.289292067049494, 'cf43': 5.290826886434751, 'cf01': 5.310117683983047, 'ce20': 5.335027365841163, 'cf33': 5.3410936366488295, 'ce02': 5.360679831514863, 'cf03': 5.37304307891982, 'cf37': 5.411977990769445, 'cd04': 5.429568329489089, 'cd07': 5.453214528040584, 'cg02': 5.503623122748463, 'cf11': 5.515077987338513, '

In [131]:
print(accuracy(sorted_perp_test_files,11.70))

99.09


## 1.4
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

I found from above code and experiment that add lambda/k smoothing easier to implement and gives better results i.e. higher accuracy as compared to linear interpolation.

However theoretically and from intuition, interpolation should provide much better results as it considers better range of n-gram probabilities.

Interpolation can deal with token not found in vocabulary condition much better than add lambda method as in add lambda method we are assigning very low value to the missing token.

[Your text here.]